In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta

In [2]:
DIR_CSV = "C:\\Users\\tora2\\IdeaProjects\\LifeBehavior\\logs\\behavior_log.csv"

In [3]:
BEHAVIOR_TYPE_TO_ACTIVITY = {
    "SLEEP": "睡眠",
    "EATING": "食事",
    "PERSONAL_CARE": "身のまわりの用事",
    "MEDICAL_CARE": "療養・静養",
    "WORK": "仕事",
    "WORK_SOCIALIZING": "仕事のつきあい",
    "SCHOOL_ACTIVITIES": "授業・学内の活動",
    "STUDY_OUTSIDE": "学校外の学習",
    "HOUSEWORK": "炊事・掃除・洗濯",
    "SHOPPING": "買い物",
    "CHILDCARE": "子どもの世話",
    "HOUSEHOLD_TASKS": "家庭雑事",
    "COMMUTING": "通勤",
    "SCHOOL_COMMUTING": "通学",
    "SOCIAL_PARTICIPATION": "社会参加",
    "SOCIALIZING": "会話・交際",
    "SPORTS": "スポーツ",
    "RECREATION": "行楽・散策",
    "HOBBIES": "趣味・娯楽・教養",
    "MEDIA_CONSUMPTION": "マスメディア接触",
    "REST": "休息"
}
# 各アクティビティに対して色を指定
COLOR_MAP = {
    '食事': '#5DAE8B',  # 濃いめの柔らかいグリーン
    '授業・学内の活動': '#E59866',  # 濃いめの柔らかいオレンジ
    '買い物': '#F7DC6F',  # 濃いめのイエロー
    '会話・交際': '#5499C7',  # 濃いめの柔らかいブルー
    'マスメディア接触': '#AF7AC5',  # 濃いめのパープル
    '仕事': '#F4D03F',  # 濃いライトゴールド
    '療養・静養': '#EC7063',  # 濃いめのピンク
    '子どもの世話': '#5DADE2',  # 濃いライトブルー
    '行楽・散策': '#DC7633',  # 濃いめのオレンジ
    '休息': '#58D68D',  # 濃いめのグリーン
    '睡眠': '#28B463',  # 濃いめのグリーン
    '通勤': '#85C1E9',  # 濃いめのブルー
    'スポーツ': '#F1C40F',  # 濃いめのライトイエロー
    '身のまわりの用事': '#E74C3C',  # 濃いめのレッド
    '家庭雑事': '#48C9B0',  # 濃いめのアクア
    '通学': '#A569BD',  # 濃いラベンダー
    '炊事・掃除・洗濯': '#F1948A',  # 濃いめのピンクベージュ
    '趣味・娯楽・教養': '#F5B041',  # 濃いゴールド
    '仕事のつきあい': '#85929E',  # 濃いめのグレー
    '社会参加': '#F8C471',  # 濃いめのライトオレンジ
    '学校外の学習': '#76D7C4'  # 濃いめのアクア
}
ACTIVITY_ORDERING = [
            "睡眠", "食事", "身のまわりの用事", "療養・静養", "仕事", "仕事のつきあい", "授業・学内の活動",
            "学校外の学習", "炊事・掃除・洗濯", "買い物", "子どもの世話", "家庭雑事", "通勤", "通学", "社会参加",
            "会話・交際", "スポーツ", "行楽・散策", "趣味・娯楽・教養", "マスメディア接触", "休息"]

def parse_current_time_to_datetime(time_str):
    # "0/00:00:00" 形式の文字列をパース
    day_part, time_part = time_str.split('/')  # 例: "0", "00:00:00"
    hours, minutes, seconds = map(int, time_part.split(':'))  # 時、分、秒をintに変換
    days = int(day_part)  # 日数部分をintに変換

    # 基準日（例えば、今日の日付）からの timedelta を計算
    base_date = datetime(2024, 9, 1)  # 基準日を設定（任意の日付を使用）
    delta = timedelta(days=days, hours=hours, minutes=minutes, seconds=seconds)
    
    # 基準日に timedelta を加えて最終的な datetime を計算
    result_datetime = base_date + delta
    return result_datetime

In [4]:
result = pd.read_csv(DIR_CSV).replace(BEHAVIOR_TYPE_TO_ACTIVITY)
weekday = result.query("CurrentDay not in ('SUNDAY','SATURDAY')").drop(columns=["CurrentDay"])
saturday = result.query("CurrentDay == 'SATURDAY'").drop(columns=["CurrentDay"])
sunday = result.query("CurrentDay == 'SUNDAY'").drop(columns=["CurrentDay"])
allday = result.drop(columns=["CurrentDay"])

In [5]:
df_grouped = pd.melt(allday, id_vars=["CurrentTime"], var_name="Human", value_name="Activity").groupby(['CurrentTime', 'Activity']).size().unstack(fill_value=0)
df_percentage = df_grouped.div(df_grouped.sum(axis=1), axis=0)
result_cleaned = df_percentage.reset_index().melt(id_vars=["CurrentTime"],value_name="Rate")
result_cleaned["Time"] = result_cleaned["CurrentTime"].apply(parse_current_time_to_datetime)
result_cleaned = result_cleaned.sort_values("Time")
result_cleaned['DayOfWeek'] = result_cleaned['Time'].dt.strftime('%a')  # 曜日 (Mon, Tue, etc.)
# 横軸に日付と曜日を表示するようにするために、日時 + 曜日の文字列を作成
result_cleaned['TimeWithDay'] = result_cleaned['Time'].dt.strftime('%m/%d %H:%M (%a)')

In [6]:
# 目盛りを間引いて設定するため、例えば1日ごとに間引いて設定
tick_step = 96  # 15分間隔でのタイムティックで96が1日分に相当（24時間 ÷ 0.25時間）
tickvals = result_cleaned['Time'][::tick_step]  # 1日ごとに目盛りを表示
ticktext = result_cleaned['TimeWithDay'][::tick_step]  # 1日ごとの時間と曜日

# 折れ線グラフを作成
fig = go.Figure()

# 算出データの実線グラフ
for activity in ACTIVITY_ORDERING:
    filtered_df = result_cleaned[result_cleaned['Activity'] == activity]
    fig.add_trace(go.Scatter(
        x=filtered_df['Time'],  # 時間軸はdatetimeのTime列
        y=filtered_df['Rate'], 
        mode='lines', 
        name=f'{activity} (統計値)', 
        line=dict(dash='solid', color=COLOR_MAP.get(activity))
    ))

# グラフのレイアウトを更新
fig.update_layout(
    title=f"推定遷移確率によるマイクロシミュレーションの結果",
    xaxis_title="時間",
    yaxis_title="割合",
    legend_title="活動",
    template="plotly",
    xaxis=dict(
        tickmode='array',  # 固定された値で目盛りを表示
        tickvals=tickvals,  # 表示したいタイムティック
        ticktext=ticktext,  # 曜日を含めた目盛りテキスト
    )
)

# 結果をHTMLで保存
fig.write_html(f"C:/Users/tora2/downloads/SOARS生活行動モデル_28日全体プロット_with_weekday.html")